# Construir nuestro primer bot RAG - Habilidad: hablar con el motor de búsqueda

Ya tenemos todos los bloques para construir nuestro primer Bot que "hable con mis datos". Estos bloques son:

1) Un motor híbrido (texto y vector) bien indexado con mis datos en trozos -> Azure AI Search
2) Un buen LLM python framework para construir LLM Apps -> LangChain
3) Modelos OpenAI GPT de calidad que entiendan el lenguaje y sigan instrucciones -> GPT3.5 y GPT4
4) Una base de datos de memoria persistente -> CosmosDB

Sólo nos falta una cosa: **Agentes**.

En este Cuaderno introducimos el concepto de Agentes y lo utilizamos para construir nuestro primer bot RAG.

In [ ]:
import os
import random
import asyncio
from typing import Dict, List
from concurrent.futures import ThreadPoolExecutor
from typing import Optional, Type

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import ConfigurableField, ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

#custom libraries that we will use later in the app
from common.utils import  GetDocSearchResults_Tool
from common.prompts import AGENT_DOCSEARCH_PROMPT

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

from dotenv import load_dotenv
load_dotenv("credentials.env")


In [ ]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

## Introducción: Agentes

La implementación de los agentes se inspira en dos artículos: el de [MRKL Systems](https://arxiv.org/abs/2205.00445) (pronunciado "milagro" 😉) y el de [ReAct](https://arxiv.org/abs/2210.03629).

Los agentes son una forma de aprovechar la capacidad de los LLM para entender y actuar en función de las instrucciones. En esencia, un Agente es un LLM al que se le ha dado una indicación inicial muy inteligente. La indicación le dice al LLM que descomponga el proceso de respuesta a una consulta compleja en una secuencia de pasos que se resuelven de uno en uno.

Los agentes se vuelven realmente interesantes cuando los combinamos con "expertos", introducidos en el documento MRKL. Un ejemplo sencillo: un agente puede no tener la capacidad inherente de realizar cálculos matemáticos de forma fiable por sí mismo. Sin embargo, podemos introducir un experto, en este caso una calculadora, experta en cálculos matemáticos. Ahora, cuando necesitemos realizar un cálculo, el Agente puede llamar al experto en lugar de intentar predecir el resultado por sí mismo. Este es en realidad el concepto detrás de [ChatGPT Pluggins](https://openai.com/blog/chatgpt-plugins).

En nuestro caso, para resolver el problema "Cómo construyo un bot inteligente que hable con mis datos", necesitamos este enfoque REACT/MRKL, en el que necesitamos instruir al LLM que necesita usar 'expertos/herramientas' para leer/cargar/entender/interactuar con una fuente de datos en particular.

Creemos entonces un Agente que interactúe con el usuario y utilice una Herramienta para obtener la información del Buscador.


#### 1. Comenzamos definiendo la Herramienta/Experto

Las herramientas son funciones que un agente puede invocar. Si no le das al agente acceso a un conjunto correcto de herramientas, nunca podrá cumplir los objetivos que le asignes. Si no describes bien las herramientas, el agente no sabrá utilizarlas correctamente.

In [ ]:
index1_name = "cogsrch-index-files"
indexes = [index1_name]

Tenemos que convertir el objeto Retreiver en un objeto Tool ("el experto"). Echa un vistazo a la herramienta `GetDocSearchResults_Tool` en `utils.py`.

Declarar las herramientas que utilizará el agente

In [ ]:
tools = [GetDocSearchResults_Tool(indexes=indexes, k=5, reranker_th=1, sas_token=os.environ['BLOB_SAS_TOKEN'])]

#### 2. Definir el LLM a utilizar

In [ ]:
COMPLETION_TOKENS = 1500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True)

#### 3. Vincular herramientas al LLM

Los modelos OpenAI más recientes (1106 y posteriores) se han ajustado para detectar cuándo se debe llamar a una o más funciones y responder con las entradas que se deben pasar a la(s) función(es). En una llamada a la API, puedes describir funciones y hacer que el modelo elija de forma inteligente la salida de un objeto JSON que contenga argumentos para llamar a estas funciones. El objetivo de las API de las herramientas de OpenAI es devolver de forma más fiable llamadas a funciones válidas y útiles que lo que puede hacerse utilizando una API genérica de completado de texto o chat.

OpenAI denomina **funciones** a la capacidad de invocar una única función, y [**herramientas**](https://platform.openai.com/docs/guides/function-calling) a la capacidad de invocar una o más funciones.

> La API de OpenAI ha dejado obsoletas las funciones en favor de las herramientas. La diferencia entre ambas es que la API de herramientas permite al modelo solicitar que se invoquen varias funciones a la vez, lo que puede reducir los tiempos de respuesta en algunas arquitecturas. Se recomienda utilizar el agente de herramientas para los modelos OpenAI.

Hacer que un LLM llame a múltiples herramientas al mismo tiempo puede acelerar enormemente los agentes si hay tareas que se ven asistidas al hacerlo. Afortunadamente, las versiones 1106 y posteriores de los modelos OpenAI soportan llamadas a funciones paralelas, lo que necesitaremos para asegurarnos de que nuestro bot inteligente tiene un buen rendimiento.

##### **De ahora en adelante y para el resto de los cuadernos, vamos a utilizar la API de herramientas de OpenAI para llamar a nuestros expertos/herramientas**.

Para pasar nuestras herramientas al agente, sólo tenemos que formatearlas al [formato de herramientas OpenAI](https://platform.openai.com/docs/api-reference/chat/create) y pasarlas a nuestro modelo. (Al unir las funciones, nos aseguramos de que se pasen cada vez que se invoque el modelo).

In [ ]:
# Bind (attach) the tools/functions we want on each LLM call

llm_with_tools = llm.bind_tools(tools)

# Let's also add the option to configure in real time the model we want

llm_with_tools = llm_with_tools.configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gpt35",
    gpt4=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True),
)

#### 4. Definir la pregunta del sistema

Debido a que OpenAI Function Calling está afinado para el uso de herramientas, apenas necesitamos instrucciones sobre cómo razonar, o cómo el formato de salida. Sólo tendremos dos variables de entrada: `question` y `agent_scratchpad`. `question` debe ser una cadena que contenga el objetivo del usuario. El `agent_scratchpad` debe ser una secuencia de mensajes que contenga las invocaciones previas a las herramientas del agente y sus correspondientes salidas.

Consigue que el prompt use `AGENT_DOCSEARCH_PROMPT` - ¡puedes modificarlo en `prompts.py`! ¡Compruébalo!
Se parece a esto:

```python
AGENT_DOCSEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", CUSTOM_CHATBOT_PREFIX  + DOCSEARCH_PROMPT_TEXT),
        MessagesPlaceholder(variable_name='history', optional=True),
        ("human", "{question}"),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)
```

In [ ]:
prompt = AGENT_DOCSEARCH_PROMPT

#### 5. Crear el agente

La idea central de los agentes es utilizar un modelo de lenguaje para elegir una secuencia de acciones a realizar. En las cadenas, la secuencia de acciones está codificada (en código). En los agentes, se utiliza un modelo de lenguaje como motor de razonamiento para determinar qué acciones realizar y en qué orden.

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "question": lambda x: x["question"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

O , lo que es equivalente, LangChain tiene una clase que hace exactamente el código de la celda de arriba: `create_openai_tools_agent`

```python
agent = create_openai_tools_agent(llm, tools, prompt)
```

Crear un agente ejecutor pasando el agente y las herramientas

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

Darle memoria

In [ ]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos

Como cosmosDB necesita dos campos (un id y una partición), y RunnableWithMessageHistory toma por defecto un único identificador para la memoria (session_id), necesitamos utilizar el parámetro `history_factory_config` y definir las múltiples claves para la clase de memoria

In [ ]:
userid_spec = ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        )
session_id = ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        )

In [ ]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[userid_spec,session_id]
)

In [ ]:
# configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
config

#### 6.Ejecutar el Agent!

In [ ]:
%%time
agent_with_chat_history.invoke({"question": "Hi, I'm Pablo Marin. What's yours"}, config=config)

In [ ]:
printmd(agent_with_chat_history.invoke(
    {"question": "What are some examples of reinforcement learning?"}, 
    config=config)["output"])

In [ ]:
printmd(agent_with_chat_history.invoke(
        {"question": "Interesting, Tell me more about this"},
        config=config)["output"])

In [ ]:
printmd(agent_with_chat_history.invoke({"question": "Thhank you!"}, config=config)["output"])

#### Importante: hay una limitación de GPT3.5, una vez que empezamos a añadir preguntas largas, y contextos largos y respuestas minuciosas, o el agente hace múltiples búsquedas para preguntas de varios pasos, ¡nos quedamos sin espacio!

Esto se puede minimizar
- Avisos del sistema más cortos
- Trozos más pequeños (menos de los 5000 caracteres por defecto)
- Reduciendo topK para traer trozos menos relevantes

Sin embargo, en última instancia, está sacrificando la calidad para que todo funcione con GPT3.5 (modelo más barato y más rápido)

### Añadamos más cosas que hemos aprendido hasta ahora: selección LLM dinámica de GPT4 y streaming asíncrono.

In [ ]:
agent = create_openai_tools_agent(llm_with_tools.with_config(configurable={"model": "gpt4"}), tools, prompt) # We select now GPT-4
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
agent_with_chat_history = RunnableWithMessageHistory(agent_executor,get_session_history,input_messages_key="question", 
                                                     history_messages_key="history", history_factory_config=[userid_spec,session_id])

En cuadernos anteriores se utilizaba la función `.stream()` del ejecutable para transmitir los tokens. Sin embargo, si necesitas transmitir tokens individuales desde el agente o pasos superficiales que ocurren dentro de las herramientas, necesitarás usar una combinación de `Callbacks` y `.astream()` O la nueva API `astream_events` (beta).

Utilicemos aquí la API astream_events para transmitir los siguientes eventos:

    Agente Inicio con entradas
    Herramienta Inicio con entradas
    Herramienta Final con salidas
    Transmitir la respuesta final del agente token a token
    Agente Final con salidas

In [ ]:
QUESTION = "Tell me more about your last answer, search again multiple times and provide a deeper explanation"

In [ ]:
async for event in agent_with_chat_history.astream_events(
    {"question": QUESTION}, config=config, version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (event["name"] == "AgentExecutor"):
            print( f"Starting agent: {event['name']}")
    elif kind == "on_chain_end":
        if (event["name"] == "AgentExecutor"):  
            print()
            print("--")
            print(f"Done agent: {event['name']}")
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        # Empty content in the context of OpenAI means that the model is asking for a tool to be invoked.
        # So we only print non-empty content
        if content:
            print(content, end="")
    elif kind == "on_tool_start":
        print("--")
        print(f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}")
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        # print(f"Tool output was: {event['data'].get('output')}")
        print("--")

#### Nota: Intenta ejecutar esta última pregunta con GPT3.5 y verás como te quedas sin espacio para tokens en el LLM

# Resumen

Acabamos de construir nuestro primer RAG BOT!.

- Hemos aprendido que **Agentes + Herramientas son la mejor manera de construir Bots**. <br>
- Convertimos el Azure Search retriever en una Herramienta usando la función `GetDocSearchResults_Tool` en `utils.py`.
- Aprendimos sobre la API de eventos (Beta), una forma de transmitir la respuesta de los agentes
- Aprendimos que para respuestas completas y de calidad nos quedaremos sin espacio con GPT3.5. GPT4 será entonces necesario.
